In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)

css_styling()

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, MaxPool2D, ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adadelta

In [4]:
(X_train, y_train), (X_test, y_test)  = cifar100.load_data()

X_train.shape, y_train.shape

169001437/169001437 [==============================] - 36s 0us/step


((50000, 32, 32, 3), (50000, 1))

In [5]:
n, img_rows, img_cols, channels = X_train.shape

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, channels)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, channels)

input_shape = (img_rows, img_cols, channels)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1]
num_pixels = img_rows * img_cols

### Building the LeNet Replica model

In [7]:
from tensorflow.keras.layers import Layer

In [6]:
class LenetBlock(Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.conv_layer = Conv2D(filters=filters, kernel_size=(5, 5), padding="same")
        self.maxpool = MaxPool2D(pool_size = (2, 2), strides = (2, 2))
        self.relu = ReLU()
        
    def call(self, inputs):
        x = self.conv_layer(inputs)
        x = self.relu(x)
        x = self.maxpool(x)
        return x

In [7]:
# create model
inputs = Input(shape=input_shape)

l1 = LenetBlock(filters=6)(inputs)
l2 = LenetBlock(filters=16)(l1)
l3 = LenetBlock(filters=120)(l2)

flat = Flatten()(l3)

d1 = Dense(120)(flat)
act1 = ReLU()(d1)
d2 = Dense(84)(act1)
act2 = ReLU()(d2)
d3 = Dense(num_classes)(act2)
outputs = Activation("softmax")(d3)

lenet_model = Model(inputs=inputs, outputs=outputs)

In [8]:
lenet_model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              metrics = ['accuracy'])
    
lenet_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 lenet_block (LenetBlock)    (None, 16, 16, 6)         456       
                                                                 
 lenet_block_1 (LenetBlock)  (None, 8, 8, 16)          2416      
                                                                 
 lenet_block_2 (LenetBlock)  (None, 4, 4, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 120)               230520    
                                                                 
 re_lu_3 (ReLU)              (None, 120)               0     

In [9]:
batch_size = 32
epochs = 10

history = lenet_model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          shuffle=True)

Epoch 1/10
1563/1563 [==============================] - 59s 35ms/step - loss: 2.2975 - accuracy: 0.1266 - val_loss: 2.2933 - val_accuracy: 0.1349
Epoch 2/10
1563/1563 [==============================] - 65s 42ms/step - loss: 2.2895 - accuracy: 0.1330 - val_loss: 2.2854 - val_accuracy: 0.1343
Epoch 3/10
1563/1563 [==============================] - 57s 36ms/step - loss: 2.2811 - accuracy: 0.1359 - val_loss: 2.2764 - val_accuracy: 0.1414
Epoch 4/10
1563/1563 [==============================] - 53s 34ms/step - loss: 2.2709 - accuracy: 0.1536 - val_loss: 2.2651 - val_accuracy: 0.1783
Epoch 5/10
1563/1563 [==============================] - 52s 33ms/step - loss: 2.2586 - accuracy: 0.1818 - val_loss: 2.2516 - val_accuracy: 0.1943
Epoch 6/10
1563/1563 [==============================] - 56s 36ms/step - loss: 2.2439 - accuracy: 0.1976 - val_loss: 2.2356 - val_accuracy: 0.2080
Epoch 7/10
1563/1563 [==============================] - 53s 34ms/step - loss: 2.2259 - accuracy: 0.2121 - val_loss: 2.2162 -

In [ ]:
lenet_model.save("CIFAR_LeNet.h5")
scores = lenet_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

### Loading data

In [32]:
from torchvision.datasets import CelebA
from torchvision.transforms import Compose, Normalize, ToTensor

In [33]:
transform = Compose([
    ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [53]:
train = CelebA(
    root="./celeba/celeba",
    split="train",
    transform=transform,
    download=True,
)

validation = CelebA(
    root="celeba",
    split="validation",
    transform=transform,
    download=False,
)

test = CelebA(
    root="celeba",
    split="test",
    transform=transform,
    download=False,
)

RuntimeError: The daily quota of the file img_align_celeba.zip is exceeded and it can't be downloaded. This is a limitation of Google Drive and can only be overcome by trying again later.

### Building AlexNet model

TODO

### Building ResNet Model

In [9]:
from tensorflow.keras.applications.resnet import ResNet50

In [10]:
resnet = ResNet50(include_top=True, weights="imagenet")

102967424/102967424 [==============================] - 18s 0us/step


In [28]:
resnet.compile(optimizer=Adadelta(), loss="categorical_crossentropy", metrics=["accuracy"])

In [30]:
resnet.evaluate(x=X_test, y=y_test)

ValueError: in user code:

    File "C:\Users\Harry\anaconda3\lib\site-packages\keras\engine\training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\Harry\anaconda3\lib\site-packages\keras\engine\training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Harry\anaconda3\lib\site-packages\keras\engine\training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\Harry\anaconda3\lib\site-packages\keras\engine\training.py", line 1756, in test_step
        y_pred = self(x, training=False)
    File "C:\Users\Harry\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Harry\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of

In [25]:
from tensorflow.keras.layers import ZeroPadding2D, BatchNormalization

In [26]:
class ConvNorm(Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.conv = Conv2D(filters, kernel_size=(3, 3), padding="same")
        self.batchnorm = BatchNormalization()
        self.activation = ReLU()
        
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.batchnorm(x)
        x = self.activation(x)
        
        return x


class ResnetBlock(Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.conv_layer = Conv2D(filters=filters, kernel_size=(5, 5), padding="same")
        self.maxpool = MaxPool2D(pool_size = (2, 2), strides = (2, 2))
        self.relu = ReLU()
        
    def call(self, inputs):
        x = self.conv_layer(inputs)
        x = self.relu(x)
        x = self.maxpool(x)
        return x
